In [4]:
import json

repo_profiles = {}
with open('repo_profiles.json') as rj:
    for rl in rj.readlines():
        line = rl.split('\t')
        repo = line[0]
        profile = json.loads(line[1])
        repo_profiles[repo] = profile

team_profiles = []
with open('team_profiles.json') as tj:
    for tl in tj.readlines():
        line = tl.split('\t')
        team = json.loads(line[0])
        profile = json.loads(line[1])
        team_profiles.append([team,profile])

In [ ]:
import numpy as np
from priority

team_recommendations = []
for repo in repo_profiles:
    rec = 
    for t_p in team_profiles:
        